# Brainstorm Proyecto 1 - DP

Se plantean dos posibles enfoques. 

1. Uno donde se parametrizan en una recurrencia el movimiento de Indiana, Marion y Salah (Dicha recurrencia será denomoniada $P$)
2. Un conjunto de tres recurrencias que modelan el comportamiento de cada personaje y que son invocadas de acuerdo a distintas prioridades de recorrido para cada una de las personas.

Antes de eso, se definen las siguientes entradas y salidas

| E/S | Nombre | Tipo | Descripción |
|-|-|-|-|
|E| A | Array[0,R)[0,C) of **int** | Matriz con la representación de la cantidad de reliquias y maldiciones de la pirámide|
|S| s | **int** | Cantidad máxima de reliquias que pueden obtener los tres personajes |

## Enfoque de única recurrencia

Tanto Marion, como Indiana y Salah pueden moverse en un espacio de $C \times \left\lfloor \dfrac{R}{2} \right\rfloor$. Por consiguiente, una estructura que pueda almacenar la cantidad maxima de reliquias obtenidas para cada combinacion de posiciones deberá tener una forma de $C \times C \times C \times \left\lfloor \dfrac{R}{2} \right\rfloor$

Para este enfoque tendremos en cuenta los siguientes puntos:

1. Todas las casillas invalidas deberán tener un valor de $-\infty$ para aquellas configuraciones invalidas (Ejemplo: Aquellas casillas con maldiciones y aquellas que en la primera y ultima fila que no correspondan al punto de partida de los personajes) 
2. La casilla inicial de cada personaje debe iniciar con cero reliquias.
3. Los demás casos recursivos representan todas las cobinaciones posibles de los movimientos de los personajes

$$ P(i,m,s,r) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r = 0 \wedge i=0 \wedge m=C-1 \wedge s = \left\lfloor \frac{C}{2} \right\rfloor \\
        -\infty & & \text{si} & & A[r][i]=-1 \vee A[r][m]=-1 \vee A[r][s]=-1 \vee \left(r=0 \wedge i>0  \wedge m < C-1 \wedge s \neq \left\lfloor \frac{C}{2} \right\rfloor \right)\\
        ... 
    \end{array}
    \right.
$$

**¿Que problema tiene?** Los indices i,m,s van los tres desde $0$ hasta $C-1$ y el indice r va desde $0$ hasta $\left\lfloor \dfrac{R}{2} \right\rfloor$. Esto significa que la complejidad tanto temporal como espacial de este algoritmo será de orden $O(C^3 R)$. Lo cual se convierte en un problema muy costoso computacionalmente hablando

## Enfoque de muchas recurrencias

Para este enfoque, diseñaremos una recurrencia para cada personaje. $I$ estara asociada a Indiana, $M$ estara asociada a Marion y $S$ estara asociada a Salah

$$ I(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r=0 \wedge i = 0 \\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r=0 \wedge i > 0) \\
        \max (I(r-1,i),I(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge i = 0 \\
        \max (I(r-1,i),I(r-1,i-1)) + A[r][i] & & \text{si} & & r>0 \wedge i = C-1 \\
        \max (I(r-1,i),I(r-1,i-1),I(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

$$ M(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r=0 \wedge i = C-1 \\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r=0 \wedge i < C-1) \\
        \max (M(r-1,i),M(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge i = 0 \\
        \max (M(r-1,i),I(r-1,i-1)) + A[r][i] & & \text{si} & & r>0 \wedge i = C-1 \\
        \max (M(r-1,i),I(r-1,i-1),M(r-1,i+1)) + A[r][i] & & \text{si} & & r>0 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

$$ S(r,i) = \left\{
    \begin{array}{}
        0 & & \text{si} & & r= R-1 \wedge i = \lfloor \frac{C}{2} \rfloor\\
        -\infty & & \text{si} & &  A[r][i] = -1 \vee (r = R-1 \wedge i \neq \lfloor \frac{C}{2} \rfloor) \\
        \max(S(r+1,i),S(r+1,i+1)) & & \text{si} & & r < R-1 \wedge i = 0 \\
        \max(S(r+1,i),S(r+1,i-1)) & & \text{si} & & r < R-1 \wedge i = C-1 \\
        \max(S(r+1,i),S(r+1,i-1),S(r+1,i+1)) & & \text{si} & & r < R-1 \wedge 0 < i < C-1 \\
    \end{array}
    \right.
$$

In [83]:
import numpy as np

def IndianaPath(A):
    R,C = A.shape
    I = np.zeros(shape=(R,C),dtype=int)
    inf = int(1e5)
    for r in range(0,R):
        for i in range(0,C):
            if r==0 and i==0: I[r,i] = 0
            elif A[r][i]==-1 or (r==0 and i>0): I[r,i] = -inf
            elif r>0 and i==0: I[r,i] = max(I[r-1,i],I[r-1,i+1]) + A[r][i]
            elif r>0 and i==C-1: I[r,i] = max(I[r-1,i],I[r-1,i-1]) + A[r][i]
            elif r>0 and 0<i<C-1: I[r,i] = max(I[r-1,i],I[r-1,i-1],I[r-1,i+1]) + A[r][i]
    row = int(np.floor(R/2))
    max_ , c = I[row,0], 0
    for column in range(1,C):
        if I[row,column] > max_:
            max_ = I[row,column]
            c = column
    if max_ < 0: return 0 , A
    #Backtrack
    r = row
    i = c 
    while(r!=0):
        k = A[r][i]
        A[r][i] = 0
        if I[r-1,i] + k != I[r,i]:
            moves = []
            if i>0:
                moves.append((r-1,i-1))
            if i<C-1:
                moves.append((r-1,i+1))
            for row,column in moves:
                if I[row,column] + k == I[r,i]:
                    i = column
                    break
        r-=1
    return max_, A

In [84]:
import numpy as np

def MarionPath(A):
    R,C = A.shape
    M = np.zeros(shape=(R,C),dtype=int)
    inf = int(1e6)
    for r in range(0,R):
        for i in range(0,C):
            if r==0 and i==C-1: M[r,i] = 0
            elif A[r][i]==-1 or (r==0 and i<C-1): M[r,i] = -inf
            elif r>0 and i==0: M[r,i] = max(M[r-1,i],M[r-1,i+1]) + A[r][i]
            elif r>0 and i==C-1: M[r,i] = max(M[r-1,i],M[r-1,i-1]) + A[r][i]
            elif r>0 and 0<i<C-1: M[r,i] = max(M[r-1,i],M[r-1,i-1],M[r-1,i+1]) + A[r][i]
    row = int(np.floor(R/2))
    max_ , c = M[row,0], 0
    for column in range(1,C):
        if M[row,column] > max_:
            max_ = M[row,column]
            c = column
    if max_ < 0: return 0 , A
    #Backtrack
    r = row
    i = c 
    while(r!=0):
        k = A[r][i]
        A[r][i] = 0
        if M[r-1,i] + k != M[r,i]:
            moves = []
            if i>0:
                moves.append((r-1,i-1))
            if i<C-1:
                moves.append((r-1,i+1))
            for row,column in moves:
                if M[row,column] + k == M[r,i]:
                    i = column
                    break
        r-=1
    return max_, A

In [85]:
import numpy as np

def SalahPath(A):
    R,C = A.shape
    S = np.zeros(shape=(R,C),dtype=int)
    inf = int(1e6)
    for r in range(R-1,-1,-1):
        for i in range(C-1,-1,-1):
            if r==R-1 and i==int(C/2): S[r,i] = 0
            elif A[r][i]==-1 or (r==R-1 and i<C-1): S[r,i] = -inf
            elif r<R-1 and i==0: S[r,i] = max(S[r+1,i],S[r+1,i+1]) + A[r][i]
            elif r<R-1 and i==C-1: S[r,i] = max(S[r+1,i],S[r+1,i-1]) + A[r][i]
            elif r<R-1 and 0<i<C-1: S[r,i] = max(S[r+1,i],S[r+1,i-1],S[r+1,i+1]) + A[r][i]
    r = int(R/2)
    max_ , c = S[r,0], 0
    for column in range(1,C):
        if S[r,column] > max_:
            max_ = S[r,column]
            c = column
    if max_ < 0: return 0 , A
    #Backtrack
    i = c 
    while(r!=R-1):
        k = A[r][i]
        A[r][i] = 0
        if S[r+1,i] + k != S[r,i]:
            moves = []
            if i>0:
                moves.append((r+1,i-1))
            if i<C-1:
                moves.append((r+1,i+1))
            for row,column in moves:
                if S[row,column] + k == S[r,i]:
                    i = column
                    break
        r+=1
    return max_, A

In [86]:
def areDisjoint(A_original,A_copies):
    if len(A_copies)==1: return True
    R,C = A_copies[0].shape
    for r in range(1,R-1):
        for c in range(0,C):
            zeros = 0
            for A in A_copies:
                if A[r,c]==0 and A_original[r,c]!=0: zeros+=1
            if zeros>1: return False
    return True

def GetMax(A,callables):
    n = len(callables)
    A_copies = [A.copy() for _ in range(n)]
    m = np.zeros(shape=(n,),dtype=int)
    for i in range(n):
        m[i] , A_copies[i] = callables[i](A_copies[i])
    if np.any(m < 0): return float("-inf")
    if areDisjoint(A,A_copies):
        return np.sum(m)
    return max(m[i]+GetMax(A_copies[i],[callables[j] for j in range(n) if j!=i]) for i in range(n))

In [87]:
from itertools import permutations

def GetMaxPermutations(A,callables):
    m = float("-inf")
    r = float("-inf")
    for c1,c2,c3 in permutations(callables):
        A_c = A.copy()
        m1 , A_c = c1(A_c)
        m2 , A_c = c2(A_c)
        m3 , A_c = c3(A_c)
        r = m1+m2+m3
        if r > m: 
            m = r
    return m

In [88]:
import numpy as np

A = np.array(
    [
        [0,9,1,10,0],
        [-1,5,5,25,5],
        [1,5,1,5,7],
        [5,5,5,15,2],
        [55,3,0,4,1]
    ]
)
calls = 0
GetMax(A,[IndianaPath,MarionPath,SalahPath])

62

In [89]:
A = np.array(
    [
        [0,9,1,10,0],
        [-1,-1,5,-1,5],
        [1,5,1,5,7],
        [5,5,5,15,2],
        [55,3,0,4,1]
    ]
)
calls = 0
GetMax(A,[IndianaPath,MarionPath,SalahPath])

32

In [90]:
A = np.array(
    [
        [0,-1,-1,-1,0],
        [2,3,-1,5,4],
        [1,5,15,2,7],
        [-1,1,-1,2,-1],
        [-1,-1,0,-1,-1]
    ]
)
calls = 0
GetMax(A,[IndianaPath,MarionPath,SalahPath])

37

In [91]:
import random
import numpy as np

def generar_matriz_aleatoria_impar(min_lado=3, max_lado=200):
    # Asegurarse de que el mínimo y máximo son impares y están en el rango correcto
    min_lado = min_lado if min_lado % 2 != 0 else min_lado + 1
    max_lado = max_lado if max_lado % 2 != 0 else max_lado - 1
    
    # Generar un tamaño impar aleatorio entre min_lado y max_lado
    a = random.choice(range(min_lado, max_lado + 1, 2))
    b = random.choice(range(min_lado, max_lado + 1, 2))
    
    # Crear la matriz con valores aleatorios (-1 o entre 1 y 1000)
    m = np.array([[random.choice([-1, random.randint(1, 1000)]) for _ in range(b)] for _ in range(a)])
    
    m[0, 0] = 0                       # Esquina superior izquierda
    m[0, b-1] = 0                     # Esquina superior derecha
    m[a-1, b//2] = 0                  # Centro de la última fila
    
    return m

In [92]:
matrices = [generar_matriz_aleatoria_impar() for _ in range(100)]

In [93]:
%%timeit
for m in matrices:
    GetMax(m,[IndianaPath,MarionPath,SalahPath])

2.72 s ± 8.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [94]:
%%timeit
for m in matrices:
    GetMaxPermutations(m,[IndianaPath,MarionPath,SalahPath])

13.3 s ± 59.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [95]:
matrices = [generar_matriz_aleatoria_impar() for _ in range(100)]
for m in matrices:
    assert(GetMaxPermutations(m,[IndianaPath,MarionPath,SalahPath]) == GetMax(m,[IndianaPath,MarionPath,SalahPath]))